In [2]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [33]:
# General png to jpg size down scale

import os
from PIL import Image

INPUT_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/People/thumbnail"
OUTPUT_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/People/thumbnail/raw"

os.makedirs(OUTPUT_DIR, exist_ok=True)

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".webp", ".webm", ".webp")):
        input_path = os.path.join(INPUT_DIR, filename)
        output_name = os.path.splitext(filename)[0] + ".jpg"
        output_path = os.path.join(OUTPUT_DIR, output_name)

        img = Image.open(input_path)

        if img.mode != "RGB":
            img = img.convert("RGB")

        img.save(
            output_path,
            format="JPEG",
            quality=60,          # Photoshop High (8)
            subsampling=2,       # 4:2:0
            optimize=True,
            progressive=True
        )

print("Done.")

Done.


In [34]:
# thumbnails only down scale
from PIL import Image
import os

INPUT_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/People/thumbnail"
OUTPUT_DIR = f"{INPUT_DIR}/downscaled"
DOWNSAMPLE_FACTOR = 4

os.makedirs(OUTPUT_DIR, exist_ok=True)

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".webp")):
        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)

        with Image.open(input_path) as img:
            w, h = img.size
            new_size = (w // DOWNSAMPLE_FACTOR, h // DOWNSAMPLE_FACTOR)

            img_resized = img.resize(new_size, Image.LANCZOS)
            img_resized.save(output_path)

print("Downsampling complete.")

Downsampling complete.


In [31]:
import os
import requests

# LM Studio local API
LM_STUDIO_URL = "http://localhost:1234/v1/chat/completions"
MODEL_NAME = "qwen/qwen3-4b-2507"  # replace with your loaded model

def translate_to_english(arabic_text):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are a professional english translator with focus in UAE attire Poetry. "
                    "Take the following Arabic text (from a filename) and return ONLY its natural English translation. "
                    "Do not add any commentary or extra characters."
                )
            },
            {
                "role": "user",
                "content": arabic_text
            }
        ],
        "temperature": 0.1
    }

    r = requests.post(LM_STUDIO_URL, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"].strip()

# Folder containing Arabic-named images
folder = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/People"

for filename in os.listdir(folder):
    if "_" in filename:
        continue  # skip already-processed files

    name, ext = os.path.splitext(filename)
    name = name.strip()

    try:
        english = translate_to_english(name)
        # Clean English text to be safe for filenames
        english_safe = "".join(c for c in english if c.isalnum() or c in " _-").strip()
    except Exception as e:
        print(f"❌ Failed to translate {filename}: {e}")
        continue

    new_name = f"{english_safe}_{name}{ext}"
    os.rename(
        os.path.join(folder, filename),
        os.path.join(folder, new_name)
    )
    print(f"✔ {filename} → {new_name}")

✔ thumbnail → thumbnail_thumbnail
✔ temp → temp_temp
✔ محمد بن سلمان.jpg → Mohammed bin Salman_محمد بن سلمان.jpg
✔ السلطان قابوس.jpg → Sultan Qaboos_السلطان قابوس.jpg
✔ Focus → Focus_Focus
✔ فاطمة بنت مبارك الكعبي.png → Fatemah bint Mubarak al-Kaabi_فاطمة بنت مبارك الكعبي.png
✔ videos → videos_videos
✔ مشعل الاحمد الجابر ال صباح.png → Mahsa Al-Ahmad Al-Jaber Al-Sabah_مشعل الاحمد الجابر ال صباح.png
✔ سلطان هيثم بن طارق.jpg → Sultan Hitham bin Tariq_سلطان هيثم بن طارق.jpg
